# DGL图的基本概念和操作

## 异构图

实际业务中要处理的数据绝大部分是异构图。这里我们用下面这张图的异构图来作为例子演示如何构建异构的DGL图。

<img src='heterogeneous_graph.png' width=60%>

该图中包含了用户，视频这两种点，以及关注，转发，点赞，收藏四种关系。其中，用户有active(活跃）和inactive（不活跃）两种标签，用户与用户之间的关系有亲密（close）和不亲密（notclose）两种标签。

下面，我们将演示如何从数据中构建异质图。首先，我们从csv文件中读取数据，包括用户（node）信息，视频（node）信息，以及四种边（关注，转发，点赞，收藏）的信息。

In [9]:
import pandas as pd
import torch as th
import dgl
import numpy as np

df_user     = pd.read_csv('raw_data/hetero_data/user_hetero.csv')
df_video    = pd.read_csv('raw_data/hetero_data/video_hetero.csv')
df_follow   = pd.read_csv('raw_data/hetero_data/user_user_follow.csv')
df_forward  = pd.read_csv('raw_data/hetero_data/user_video_forward.csv')
df_thumb    = pd.read_csv('raw_data/hetero_data/user_video_thumb.csv')
df_save     = pd.read_csv('raw_data/hetero_data/user_video_save.csv')
print(df_user.keys())
print(df_video.keys())
df_user.head(4)

Index(['UID', 'gender', 'age', 'education', 'favorate', 'followers', 'label'], dtype='object')
Index(['VID', 'likes', 'forwards', 'timestamp'], dtype='object')


,UID,gender,age,education,favorate,followers,label
0,101,F,18,college,movie,19,inactive
1,102,M,38,college,exercise,300,active
2,103,M,19,college,music,294,inactive
3,104,M,17,high school,music,499,active


### 将节点编号从0重排

类似于同质图，我们也需要对对Node ID从0开始重新排序以满足dgl对于数据要求。注意这里，用户和视频都从0开始排序。

In [10]:
user_mapping = {idx: i for i, idx in enumerate(df_user['UID'].unique())}
num_users = len(user_mapping)

video_mapping = {idx: i for i, idx in enumerate(df_video['VID'].unique())}
num_video = len(video_mapping)

df_dict = {'forward': df_forward, 'thumb':df_thumb, 'save':df_save} # df_follow单独处理

### 根据边列表创建图结构

下面，我们创建边列表，由于图中包含关注，转发，点赞，收藏四种边，为了进行区分，我们需要对每一种边都创建单独的列表，因此我们用字典来存储。

In [11]:
edge_dict = {}
for relation, df in df_dict.items():
    src = [user_mapping[idx] for idx in df['UID']] # 起始节点
    dst = [video_mapping[idx] for idx in df['VID']] # 终止节点
    edge_dict[relation] =  (th.tensor(src), th.tensor(dst))

src = [user_mapping[idx] for idx in df_follow['UID1']] 
dst = [user_mapping[idx] for idx in df_follow['UID2']] 
edge_dict["follow"] =  (th.tensor(src), th.tensor(dst))

dgl中，我们用dgl.heterograph(）建立异质图，其输入为包含边类型和边数据的字典。

In [12]:
graph_data = {
   ('user', 'follow', 'user'): edge_dict['follow'],
   ('user', 'forward', 'video'): edge_dict['forward'],
   ('user', 'save', 'video'): edge_dict['save'],
   ('user', 'thumb', 'video'): edge_dict['thumb']
}
g = dgl.heterograph(graph_data)
print(g)

Graph(num_nodes={'user': 7, 'video': 7},
      num_edges={('user', 'follow', 'user'): 4, ('user', 'forward', 'video'): 4, ('user', 'save', 'video'): 4, ('user', 'thumb', 'video'): 8},
      metagraph=[('user', 'user', 'follow'), ('user', 'video', 'forward'), ('user', 'video', 'save'), ('user', 'video', 'thumb')])


### 处理节点和边的特征

类似于同质图，我们处理用户的feature信息

In [14]:
feature_list = []
for key in ['gender', 'education', 'favorate']: # one hot
    value = df_user[key].str.get_dummies('|').values 
    value= th.from_numpy(value).to(th.float) 
    feature_list.append(value)
    
for key in ['age', 'followers']: # normalize
    value = df_user[key].values
    value= th.from_numpy(value).to(th.float).unsqueeze(0)
    value = value/max(value[0]) # normalize
    feature_list.append(value.permute(1, 0))
    
all_feature_user = th.cat(feature_list, dim=1)
print('all_feature_user', all_feature_user.shape)

all_feature_user torch.Size([7, 9])


同上，我们处理video的feature信息。

In [15]:
feature_list = []
for key in ['likes', 'forwards', 'timestamp']:
    value = df_video[key].values
    value= th.from_numpy(value).to(th.float).unsqueeze(0)
    value = value/max(value[0]) # normalize
    feature_list.append(value.permute(1, 0))
    
all_feature_video = th.cat(feature_list, dim=1)
print(all_feature_video.shape)

torch.Size([7, 3])


### 处理节点和边的标签

In [22]:
value = df_user['label'].str.get_dummies('|').values 
value = np.where(value>0)[1]
node_label = th.from_numpy(value).to(th.float) 
print('user label',node_label)

user label tensor([1., 0., 1., 0., 0., 1., 1.])


In [23]:
value = df_follow['label'].str.get_dummies('|').values 
value = np.where(value>0)[1]
edge_label = th.from_numpy(value).to(th.float) 
print('edge label',edge_label)

edge label tensor([0., 1., 1., 0.])


以上，我们完成了读取数据、将节点编号重排、提取所有类型的边、提取节点信息，下面我们将这些数据放入图中。

建立图后，我们可以查看图中的各种信息。例如，用g.nodes['user'].data['feature']可以访问user节点的特征信息。

In [25]:
print(g.nodes('video'))
print(g.ntypes)
print(g.etypes)
print(g.canonical_etypes)
g.nodes['user'].data['feature'] = all_feature_user
g.nodes['video'].data['video_feature'] = all_feature_video
g.nodes['user'].data['label'] = node_label
g.edges['follow'].data['label'] = edge_label

tensor([0, 1, 2, 3, 4, 5, 6])
['user', 'video']
['follow', 'forward', 'save', 'thumb']
[('user', 'follow', 'user'), ('user', 'forward', 'video'), ('user', 'save', 'video'), ('user', 'thumb', 'video')]


### 保存图

In [26]:
from dgl.data.utils import save_graphs, load_graphs

hetero_local_path = 'hetero_dgl.bin'
save_graphs(hetero_local_path, [g])

### 加载图

In [27]:
graph = load_graphs(hetero_local_path)
print(graph)

([Graph(num_nodes={'user': 7, 'video': 7},
      num_edges={('user', 'follow', 'user'): 4, ('user', 'forward', 'video'): 4, ('user', 'save', 'video'): 4, ('user', 'thumb', 'video'): 8},
      metagraph=[('user', 'user', 'follow'), ('user', 'video', 'forward'), ('user', 'video', 'save'), ('user', 'video', 'thumb')])], {})
